In [ ]:
# Input data and output folders

inProdPlants = "ProductionPlants.txt"
inRobots = "Robots.txt"
inFailures = "Failures.txt"

outputPathPart1 = "outPart1/"
outputPathPart2 = "outPart2/"

In [ ]:
# Write your code here

In [ ]:
# Part 1

In [ ]:
prodPlantsRDD = sc.textFile(inProdPlants)

In [ ]:
robotsRDD = sc.textFile(inRobots)

In [ ]:
# Select production plants located in Italy
prodPlantsItalyRDD = prodPlantsRDD.filter(lambda line: line.split(",")[2] == 'Italy')

In [ ]:
# Map each line of prodPlantsRDD to a pair (PlantID, City)
def plantIdCity(line):
    fields = line.split(",")
    plantID = fields[0]
    city = fields[1]
    
    return (plantID,city)
    
planIDCityRDD = prodPlantsItalyRDD.map(plantIdCity)

In [ ]:
# Map each line of robotsRDD to a pair (PlantID, +1)
ridNoneRDD = robotsRDD.map(lambda line: (line.split(",")[1], +1))

In [ ]:
# Join planIDCityRDD with ridNoneRDD
joinPlantIDRIDOneRDD = planIDCityRDD.join(ridNoneRDD)

In [ ]:
# Each value part cointains a pair (City, +1) 
cityOneRDD = joinPlantIDRIDOneRDD.values()

In [ ]:
# Count the number of robots per city
citiesTotRobotsRDD = cityOneRDD.reduceByKey(lambda v1,v2: v1+v2)

In [ ]:
# Compute the maximum number of robots 
maxRobots = citiesTotRobotsRDD.values().reduce(lambda v1,v2: max(v1,v2))

In [ ]:
# Select the city/ies) associated with the maximum value
selectedCitiesRDD = citiesTotRobotsRDD.filter(lambda pair: pair[1]==maxRobots)

In [ ]:
selectedCitiesRDD.keys().saveAsTextFile(outputPathPart1)

In [ ]:
# Part 2

In [ ]:
failuresRDD = sc.textFile(inFailures)

In [ ]:
# Map each failure to a pair (RID, +1)  
planIDCityRDD = failuresRDD.map(lambda line: (line.split(",")[0], +1))

In [ ]:
# Compute the number of failures for each robot

In [ ]:
failuresRobotRDD = planIDCityRDD.reduceByKey(lambda v1, v2: v1+v2)

In [ ]:
# Map each robot to a pair (RID, PlantId)  
def ridPlantID(line):
    fields = line.split(",")
    rid = fields[0]
    pid = fields[1]
    
    return (rid,pid)


ridPidRDD = robotsRDD.map(ridPlantID)

In [ ]:
# Join ridPidRDD with failuresRobotRDD
joinRidPidNumFailuresRDD = ridPidRDD.join(failuresRobotRDD)

In [ ]:
# Map to pairs (PID, numFailuresRobot) and compute the maximum value for each PID
# This RDD does not contains the production plants without failures
pidMaxFailuresPerRobotRDD = joinRidPidNumFailuresRDD.values().reduceByKey(lambda v1, v2: max(v1, v2))

In [ ]:
# Map input productin plants to pair (plantId, 0)
pidZeroRDD = prodPlantsRDD.map(lambda line: (line.split(",")[0], 0))

In [ ]:
# Retrieve the production plants with a maximum number of failures per robots equal to 0
pidNoFailuresRDD = pidZeroRDD.subtractByKey(pidMaxFailuresPerRobotRDD)

In [ ]:
# The final result is the union of pidMaxFailuresPerRobotRDD and pidNoFailuresRDD
resPart2RDD = pidMaxFailuresPerRobotRDD.union(pidNoFailuresRDD)

In [ ]:
resPart2RDD.saveAsTextFile(outputPathPart2)